In [1]:
import pandas as pd
import requests
import json
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore') 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


c:\Users\Lee\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [36]:
import os
os.chdir(r'E:\제출용데이터dir')


# 1. 데이터 전처리

> ## 1.1 주거정보 전처리

> ## 1.2 제조업체 전처리

> ## 1.3 산업단지 전처리

# 2. 버스 정류장 좌표를 통해 주변 시설 구하기

>## 2.1데이터 불러오기

In [8]:
bus = pd.read_csv('./2022년_전국버스정류장 위치정보_데이터.csv')
bus.head(2)

,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명
0,ASB214010001,석근2리 입구,36.935731,127.043094,20221012050123,10001.0,34040,아산시,아산
1,ASB214015410,석근2리 입구,36.935633,127.043204,20221012050123,15410.0,34040,아산시,아산


>## 2.2 카카오 api를 통하여, 천안시 버스정류장을 필터링하기
천안시의 버스정류장을 고려하기 위해서, 천안시의 버스정류장 데이터를 필터링한다.

제공된 버스정류장의 데이터에는 위도, 경도 좌표에 부여된 도시명과 맞지 않는 부분이 있기 때문에,

카카오 api를 활용하여, 도로명 주소를 구하고 도로명주소를 활용하여 버스 정류장이 천안시에만 있는 데이터만을 추출한다.


In [9]:
addressCopy = bus.copy()
addressCopy = addressCopy.head()
apiKey='1d49e941371e685a965fba06520cda68'
headers = {'Authorization': 'KakaoAK {}'.format(apiKey) }

addressCopy['coord2address'] = None

cnt=0 
for idx in tqdm(range(len(addressCopy))):
  try:

    x = addressCopy.at[idx,'경도']
    y =  addressCopy.at[idx,'위도']
    searching ='x={x}&y={y}'.format(x=str(x),y=str(y))
    url='https://dapi.kakao.com/v2/local/geo/coord2address.json?{}'.format(searching)
    data = requests.post(url,headers=headers)
    result = json.loads(data.text)
    # print(result)
    documents = result['documents']
    
    addressCopy.at[idx,'coord2address'] = documents[0]
  except:
    cnt+=1
    if cnt%50==0:
      print('쿼리값을 못찾는중:',cnt) 

    pass

addressCopy

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:00<00:00, 13.20it/s]


,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명,coord2address
0,ASB214010001,석근2리 입구,36.935731,127.043094,20221012050123,10001.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na..."
1,ASB214015410,석근2리 입구,36.935633,127.043204,20221012050123,15410.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na..."
2,ASB214015456,오성슈퍼,36.951671,127.052784,20221012050123,15456.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na..."
3,ASB214015459,팽성초교입구,36.952036,127.053031,20221012050123,15459.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na..."
4,ASB214015461,대사리입구,36.944598,127.053092,20221012050123,15461.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na..."


In [10]:
# 제공된 버스정류장의 데이터에선 정류장번호가 ASB214010001 인 데이터의 도시명은 아산시라고 되어있지만, 카카오 api를 통해 나온 도로명 주소는 평택시로 되어있다.
# 구글 맵을 통해, 위도 경도 좌표를 넣어보면, 구글 맵또한 평택시라고 명시되어 있다.
addressCopy.loc[0,'coord2address']

# '시' 를 구분하는 region_2depth_name 데이터를 통해, 제공된 버스 정류장 데이터에 새로운 칼럼을 부여하여,
# region_2depth_name 칼럼을 통해, 천안시인 데이터만을 필터링한다.
addressCopy['region_2depth_name'] = addressCopy['coord2address'].apply(lambda elm : elm.get('address').get('region_2depth_name'))
addressCopy

{'road_address': None,
 'address': {'address_name': '경기 평택시 팽성읍 석근리 191-1',
  'region_1depth_name': '경기',
  'region_2depth_name': '평택시',
  'region_3depth_name': '팽성읍 석근리',
  'mountain_yn': 'N',
  'main_address_no': '191',
  'sub_address_no': '1',
  'zip_code': ''}}

,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명,coord2address,region_2depth_name
0,ASB214010001,석근2리 입구,36.935731,127.043094,20221012050123,10001.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na...",평택시
1,ASB214015410,석근2리 입구,36.935633,127.043204,20221012050123,15410.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na...",평택시
2,ASB214015456,오성슈퍼,36.951671,127.052784,20221012050123,15456.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na...",평택시
3,ASB214015459,팽성초교입구,36.952036,127.053031,20221012050123,15459.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na...",평택시
4,ASB214015461,대사리입구,36.944598,127.053092,20221012050123,15461.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na...",평택시


In [9]:
# 천안시의 데이터만 필터링한 후 저장하기
filtered_addressCopy = addressCopy[addressCopy['region_2depth_name']=='천안시']
filtered_addressCopy.to_pickle('./bus_coord2address.pickle')

>## 2.3 카카오 api를 통해 얻은 도로명 주소를 통해 반경 0.5km 이내에 있는 시설 종합하기

In [15]:
# ToDo. 시,구, 읍/면/동, 리 뽑는 코드

In [109]:
bus_station_Main = pd.read_pickle('./Bus_Main.pickle')
bus_station_Main.head(3)

,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명,MT1,...,HP8,PM9,주변시설의총합,coord2address,region_2depth_name,도로명주소,시,구,읍/면/동,리
0,ASB285000077,성환8리,36.921468,127.122774,20221012050123,77.0,34040,아산시,아산,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,550,"{'road_address': None, 'address': {'address_na...",천안시 서북구,충남 천안시 서북구 성환읍 성환리 675-1,천안시,서북구,성환읍,성환리
1,ASB285000078,송덕,36.923783,127.118976,20221012050123,78.0,34040,아산시,아산,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,515,"{'road_address': None, 'address': {'address_na...",천안시 서북구,충남 천안시 서북구 성환읍 우신리 24-5,천안시,서북구,성환읍,우신리
2,ASB285000079,국립축산과학원,36.925931,127.111136,20221012050123,79.0,34040,아산시,아산,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,245,"{'road_address': None, 'address': {'address_na...",천안시 서북구,충남 천안시 서북구 성환읍 신방리 1-10,천안시,서북구,성환읍,신방리


In [110]:
bus_station_cp = bus_station_Main.copy()
CateGroupCode = ['MT1','CS2','PS3','SC4','AC5','PK6','OL7','SW8','BK9','CT1','AG2','PO3','AT4','AD5','FD6','CE7','HP8','PM9']
code_df = pd.DataFrame(columns=CateGroupCode)
bus_code_df = pd.concat([bus_station_Main,code_df],axis=1)
bus_code_df

,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명,MT1,...,BK9,CT1,AG2,PO3,AT4,AD5,FD6,CE7,HP8,PM9
0,ASB285000077,성환8리,36.921468,127.122774,20221012050123,77.0,34040,아산시,아산,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ASB285000078,송덕,36.923783,127.118976,20221012050123,78.0,34040,아산시,아산,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ASB285000079,국립축산과학원,36.925931,127.111136,20221012050123,79.0,34040,아산시,아산,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ASB285000094,성환8리,36.921479,127.122505,20221012050123,94.0,34040,아산시,아산,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ASB285000095,성환터미널,36.917221,127.132732,20221012050123,95.0,34040,아산시,아산,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2309,GGB285000034,성환터미널,36.916583,127.132933,20221012043004,81034.0,34010,천안시,경기도,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2310,GGB285000035,남서울대학,36.910267,127.142200,20221012043004,81035.0,34010,천안시,경기도,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2311,GGB285500010,천안TG.단대.천안대,36.822650,127.162867,20221012043004,0.0,34010,천안시,경기도,{'documents': [{'address_name': '충남 천안시 동남구 신부...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2312,GGB285500020,천안종합터미널,36.819833,127.155733,20221012043004,0.0,34010,천안시,경기도,{'documents': [{'address_name': '충남 천안시 동남구 신부...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# ToDo. 도로명주소 -> 반경 0.5km 이내에 있는 데이터를 긁는 코드 필요
cnt=0
# 적용할 데이터프레임
# bus_code_df = 데이터프레임.copy()
bus_code_df = bus_code_df.head(2)


headers = {'Authorization': 'KakaoAK {}'.format(apiKey) }
for code in tqdm(CateGroupCode):
  for idx in tqdm(range(len(bus_code_df))):
    x = bus_code_df.at[idx,'경도']
    y = bus_code_df.at[idx,'위도']
    try:
      searching ='category_group_code={code}&x={x}&y={y}&radius={radius}'.format(code=code,x=str(x),y=str(y),radius=500)
      url='https://dapi.kakao.com/v2/local/search/category.json?{}'.format(searching)
      data=requests.post(url,headers=headers)
      result = json.loads(data.text)
      # print(result)    
      
      

      bus_code_df.at[idx,code] = result.get('meta').get('total_count')


    except:
      cnt+1
      if cnt%50 ==0:
        print('안긁히고 있는 데이터 갯수:',cnt)
      pass
  print(code,'코드 완료')
bus_code_df

  6%|▌         | 1/18 [00:00<00:02,  8.14it/s]

MT1 코드 완료


 11%|█         | 2/18 [00:00<00:01,  8.02it/s]

CS2 코드 완료


 17%|█▋        | 3/18 [00:00<00:01,  7.95it/s]

...

 94%|█████████▍| 17/18 [00:02<00:00,  7.75it/s]

HP8 코드 완료


100%|██████████| 18/18 [00:02<00:00,  7.91it/s]

PM9 코드 완료


,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명,MT1,...,BK9,CT1,AG2,PO3,AT4,AD5,FD6,CE7,HP8,PM9
0,ASB285000077,성환8리,36.921468,127.122774,20221012050123,77.0,34040,아산시,아산,0,...,2,1,2,0,0,0,8,0,1,0
1,ASB285000078,송덕,36.923783,127.118976,20221012050123,78.0,34040,아산시,아산,0,...,2,0,3,0,0,0,3,0,0,0


In [ ]:
# bus_code_df 를 "Bus_around_500_08231838 복사본" 으로 저장합니다.

In [21]:
bus_byApi = pd.read_pickle('./Bus_around_500_08231838 복사본.pickle')
bus_byApi.head(3)

,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,BK9,CT1,AG2,PO3,AT4,AD5,FD6,CE7,HP8,PM9
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,0,0,0,0,0,8,0,1,0
1,ASB285000078,송덕,36.923783,127.118976,0,2,2,1,0,2,...,0,0,0,0,0,0,3,0,0,0
2,ASB285000079,국립축산과학원,36.925931,127.111136,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
# 모든 카테고리의 수를 종합한 칼럼 category_sum 칼럼은 추후 score 칼럼을 계산할 때 사용된다
bus_byApi['category_sum'] = bus_byApi.loc[:,'MT1':].sum(axis=1)
bus_byApi.head(2)

,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,CT1,AG2,PO3,AT4,AD5,FD6,CE7,HP8,PM9,category_sum
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,0,0,0,0,8,0,1,0,17
1,ASB285000078,송덕,36.923783,127.118976,0,2,2,1,0,2,...,0,0,0,0,0,3,0,0,0,12


# 3. 버스 정류장 - 공원 사이의 거리 구하기

버스 정류장과 공원 사이의 거리를 구하여, 구간 별로 나눈 범위 중 몇개가 있는지 구한다

In [27]:
# ToDo. 거리 계산한 코드 필요

In [24]:
bus_park = pd.read_pickle('./버스정류장과공원사이의거리.pickle')
bus_park

,정류장번호,정류장명,location,공원명,공원좌표,공원관리번호,distance
0,ASB285000077,성환8리,"(36.92146807, 127.12277401)",용곡공원(용곡4),"(36.78741709, 127.1355518)",44131-00043,14.919877
1,ASB285000077,성환8리,"(36.92146807, 127.12277401)",눈들공원(용곡1),"(36.79261113, 127.1392161)",44131-00044,14.374872
2,ASB285000077,성환8리,"(36.92146807, 127.12277401)",원성소공원,"(36.81341463, 127.159059)",44131-00045,12.420054
3,ASB285000077,성환8리,"(36.92146807, 127.12277401)",원성공원(원성2),"(36.80788022, 127.1636754)",44131-00046,13.122437
4,ASB285000077,성환8리,"(36.92146807, 127.12277401)",꽃재공원(원성4),"(36.80085077, 127.1616121)",44131-00047,13.826295
...,...,...,...,...,...,...,...
765343,GGB500000184,둔포,"(36.9302833, 127.0387333)",백석공원,"(36.83303341, 127.1090661)",44133-00098,12.481557
765344,GGB500000184,둔포,"(36.9302833, 127.0387333)",두정10공원,"(36.82681344, 127.1507137)",44133-00099,15.215725
765345,GGB500000184,둔포,"(36.9302833, 127.0387333)",성성4지구 어린이공원,"(36.84053144, 127.1368524)",44133-00100,13.255707
765346,GGB500000184,둔포,"(36.9302833, 127.0387333)",문성소공원,"(36.80968397, 127.1542359)",44131-00065,16.887250


* class_A : 버스정류장과 타겟과의 거리가 0이상 0.5 이하에 속하는 것

* class_B : 버스정류장과 타겟과의 거리가 0.5초과 1 이하에 속하는 것

        (거리 단위는 km)

In [25]:
# 버스 정류장과 공원 사이의 거리가 0<=distance<=0.5, 0.5<distance<=1 의 갯수를 각각 구한다.
bus_park_class_A = bus_park[(0<=bus_park['distance'])&(bus_park['distance']<=0.5)]
bus_park_class_B = bus_park[(0.5<bus_park['distance'])&(bus_park['distance']<=1)]
bus_park_class_A.head(2)
bus_park_class_B.head(2)

,정류장번호,정류장명,location,공원명,공원좌표,공원관리번호,distance
1493,ASB285000099,삼풍아파트,"(36.9125166, 127.1309215)",성환4공원,"(36.91229381, 127.1344771)",44133-00053,0.317816
2933,ASB285000165,쌍용동일아파트,"(36.7848, 127.11328)",쌍용10공원,"(36.78655184, 127.1168556)",44133-00077,0.373709


,정류장번호,정류장명,location,공원명,공원좌표,공원관리번호,distance
785,ASB285000095,성환터미널,"(36.91722136, 127.13273235)",성환4공원,"(36.91229381, 127.1344771)",44133-00053,0.568513
962,ASB285000096,성신초등학교,"(36.9176694, 127.1277947)",성환4공원,"(36.91229381, 127.1344771)",44133-00053,0.842896


In [26]:
# 공원거리구분_A : 0.5<distance<=1 
class_A_df= pd.DataFrame(bus_park_class_A['정류장번호'].value_counts()).reset_index()
class_A_df.columns =['정류장번호','공원거리구분_A']
class_A_df.sort_values('정류장번호',ascending=True,inplace=True)
class_A_df.reset_index(drop=True,inplace=True)
class_A_df

# 공원거리구분_B : 0.5<distance<=1 


class_B_df= pd.DataFrame(bus_park_class_B['정류장번호'].value_counts()).reset_index()
class_B_df.columns =['정류장번호','공원거리구분_B']
class_B_df.sort_values('정류장번호',ascending=True,inplace=True)
class_B_df.reset_index(drop=True,inplace=True)
class_B_df

,정류장번호,공원거리구분_A
0,ASB285000099,1
1,ASB285000165,1
2,ASB285000287,1
3,ASB285000649,2
4,ASB285000650,2
...,...,...
781,GGB277102211,1
782,GGB285000034,1
783,GGB285500010,4
784,GGB285500020,6


,정류장번호,공원거리구분_B
0,ASB285000095,1
1,ASB285000096,1
2,ASB285000097,1
3,ASB285000098,1
4,ASB285000100,1
...,...,...
1008,GGB285500010,4
1009,GGB285500020,8
1010,GGB285500030,8
1011,GGB288500130,8


In [30]:
bus_station= bus_byApi.copy()
bus_station_merge_class_A = pd.merge(bus_station,class_A_df,on='정류장번호',how='left')

# 천안시의 버스정류장 데이터와, 천안시 버스정류장-공원거리구분 A 데이터가 잘 병합됬었는지 데이터 갯수를 통해 확인
print('테이블간 결합됨 :',len(bus_station)==len(bus_station_merge_class_A))

bus_station_merge_class_AB = pd.merge(bus_station_merge_class_A,class_B_df, on='정류장번호',how='left')


print('테이블간 결합됨 :',len(bus_station)==len(bus_station_merge_class_AB))

테이블간 결합됨 : True
테이블간 결합됨 : True


In [31]:
bus_station_merge_class_AB.head(1)
# 테이블 병합이후, NaN값은 0으로 채우고, 병합한 컬럼의 자료형은 int 로 변경
bus_station_merge_class_AB['공원거리구분_A'] = bus_station_merge_class_AB['공원거리구분_A'].fillna(0)
bus_station_merge_class_AB['공원거리구분_B'] = bus_station_merge_class_AB['공원거리구분_B'].fillna(0)

bus_station_merge_class_AB['공원거리구분_A'] = bus_station_merge_class_AB['공원거리구분_A'].astype('int')
bus_station_merge_class_AB['공원거리구분_B'] = bus_station_merge_class_AB['공원거리구분_B'].astype('int')

bus_station_merge_class_AB.head(1)


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,PO3,AT4,AD5,FD6,CE7,HP8,PM9,category_sum,공원거리구분_A,공원거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,0,0,8,0,1,0,17,NaN,NaN


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,PO3,AT4,AD5,FD6,CE7,HP8,PM9,category_sum,공원거리구분_A,공원거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,0,0,8,0,1,0,17,0,0


# 4. 버스 정류장과 거주지, 회사의 갯수 구하기

위의 작업과 마찬가지로, 버스정류장과 거주지,회사간의 거리를 구하여 그 거리를 구간별로 나눠 갯수를 구하는 작업을 진행합니다.


In [ ]:
# ToDo 버스 정류장과 거주기, 회사의 거리 구하는 코드 추가하기 
cmp1 = pd.read_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/제조업체/제조업체데이터0819.pickle')
cmp2 = pd.read_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/산업단지/산업단지byAPI_0817.pickle')
cmp2.rename(columns={'회사명':'회사'},inplace=True)
cmp1['회사구분'] ='제조업체'
cmp2['회사구분'] ='산업단지'
cmp1 = cmp1[['회사','byAPI','회사구분']]
cmp2 = cmp2[['회사','byAPI','회사구분']]
cmpAll = pd.concat([cmp1,cmp2],axis=0)
cmp1.head()
cmp2.head()
cmpAll.head()
cmpAll.reset_index(inplace=True,drop=True)

In [ ]:
cmpAll['x'] = cmpAll['byAPI'].apply(lambda elm: elm.get('x', {}))
cmpAll['y'] = cmpAll['byAPI'].apply(lambda elm: elm.get('y', {}))
cmpAll['x'] = cmpAll['x'].astype('float64')
cmpAll['y'] = cmpAll['y'].astype('float64')
cmpAll['location'] = pd.Series(zip(cmpAll['y'],cmpAll['x']))
cmpAll.head(2)

In [ ]:
# ToDo.  addressByAPIpickle08191010구한 코드 추가하기
house = pd.read_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/주거단지/addressByAPIpickle08191010.pickle')
house.head(2)
house_cp = house[['건물명','byAPI']]
house_cp.head(2)

In [ ]:
house_cp['x'] = house_cp['byAPI'].apply(lambda elm : elm.get('x'))
house_cp['y'] = house_cp['byAPI'].apply(lambda elm : elm.get('y'))
house_cp['x'] = house_cp['x'].astype('float64')
house_cp['y'] = house_cp['y'].astype('float64')

def create_location(row):
    return (row['y'], row['x'])

house_cp['location'] = house_cp.apply(create_location, axis=1)

cmpAll.rename(columns = {'회사':'이름','회사구분':'종류'},inplace=True)
house_cp.rename(columns = {'건물명':'이름'},inplace=True)

In [ ]:
company_house = pd.concat([cmpAll,house_cp],axis=0)
company_house

In [ ]:
bus = pd.read_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/버스정류장/around_bus_info_2km_08161152.pickle')
bus_cp = bus[['정류장번호','정류장명','위도','경도','coord2address','주변시설의총합']]
bus_cp.reset_index(drop=True, inplace=True)

In [ ]:
bus_cp['location'] = pd.Series(zip(bus_cp['위도'],bus_cp['경도']))
bus_cp.head()

In [ ]:
# 거리 구하기
# x : 경도, y : 위도
def calculate_distance(bus_cp, company_house):
    data_list = []

    for idx in tqdm(range(len(bus_cp))):
        for jdx in range(len(company_house)):
            
            distance = geopy.distance.distance(bus_cp.at[idx,'location'],company_house.at[jdx,'location']).km

            data = {'정류장번호':bus_cp.at[idx,'정류장번호'],'정류장명':bus_cp.at[idx,'정류장명'],
            'coord2address':bus_cp.at[idx,'coord2address'],'주변시설의총합':bus_cp.at[idx,'주변시설의총합'],
            'location':bus_cp.at[idx,'location'],'이름':company_house.at[jdx,'이름'],'byAPI':company_house.at[jdx,'byAPI'],
            'house_loc':company_house.at[jdx,'location'],'종류':company_house.at[jdx,'종류'],'distance':distance}
            
            data_list.append(data)

    data_df = pd.DataFrame(data_list)
    return data_df
    
company_house.reset_index(drop=True,inplace=True)
bus_Around_company_house = calculate_distance(bus_cp,company_house)
bus_Around_company_house

In [38]:
bus_CH = pd.read_pickle('./bus_Around_company_house.pickle')
bus_CH.head(2)

,정류장번호,정류장명,coord2address,주변시설의총합,location,이름,byAPI,house_loc,종류,distance
0,ASB285000077,성환8리,"{'road_address': None, 'address': {'address_na...",550,"(36.92146807, 127.12277401)",(주)동아수지산업,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...,"(36.8656506084092, 127.103428417573)",제조업체,6.429916
1,ASB285000077,성환8리,"{'road_address': None, 'address': {'address_na...",550,"(36.92146807, 127.12277401)",KY컴퍼니,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...,"(36.8662931269303, 127.104524303521)",제조업체,6.335470


In [39]:
# A, B 로 구간 나누기
bus_CH_A = bus_CH[(0<=bus_CH['distance'])&(bus_CH['distance']<=0.5)]
bus_CH_B = bus_CH[(0.5<bus_CH['distance'])&(bus_CH['distance']<=1)]



In [40]:
# # class_A : 버스 정류장과 집,회사 사이의 거리가 0<=distance<=0.5
bus_CH_class_A_df= pd.DataFrame(bus_CH_A['정류장번호'].value_counts()).reset_index()
bus_CH_class_A_df.columns =['정류장번호','CH거리구분_A']
bus_CH_class_A_df.sort_values('정류장번호',ascending=True,inplace=True)
bus_CH_class_A_df.reset_index(drop=True,inplace=True)
bus_CH_class_A_df.head(2)

# # class_B : 버스 정류장과 집,회사 사이의 거리가 0.5< distance<=1
bus_CH_class_B_df= pd.DataFrame(bus_CH_B['정류장번호'].value_counts()).reset_index()
bus_CH_class_B_df.columns =['정류장번호','CH거리구분_B']
bus_CH_class_B_df.sort_values('정류장번호',ascending=True,inplace=True)
bus_CH_class_B_df.reset_index(drop=True,inplace=True)
bus_CH_class_B_df.head(2)

,정류장번호,CH거리구분_A
0,ASB285000077,5
1,ASB285000078,11


,정류장번호,CH거리구분_B
0,ASB285000077,25
1,ASB285000078,15


In [41]:
# 테이블을 병합하기위해, copy본과 결합
bus_station_merge_class_AB_cp = bus_station_merge_class_AB.copy()

bus_station_merge_CH_class_A = pd.merge(bus_station_merge_class_AB_cp,bus_CH_class_A_df,on='정류장번호',how='left')

# 천안시의 버스정류장 데이터와, 천안시 버스정류장-공원거리구분 A 데이터가 잘 병합됬었는지 데이터 갯수를 통해 확인
print('테이블간 결합됨',len(bus_station)==len(bus_station_merge_CH_class_A))

bus_station_merge_CH_class_AB = pd.merge(bus_station_merge_CH_class_A,bus_CH_class_B_df, on='정류장번호',how='left')

print('테이블간 결합됨',len(bus_station)==len(bus_station_merge_CH_class_AB))

bus_station_merge_CH_class_AB.head(1)
# 테이블 병합이후, NaN값은 0으로 채우고, 병합한 컬럼의 자료형은 int 로 변경
bus_station_merge_CH_class_AB['CH거리구분_A'] = bus_station_merge_CH_class_AB['CH거리구분_A'].fillna(0)
bus_station_merge_CH_class_AB['CH거리구분_B'] = bus_station_merge_CH_class_AB['CH거리구분_B'].fillna(0)

bus_station_merge_CH_class_AB['CH거리구분_A'] = bus_station_merge_CH_class_AB['CH거리구분_A'].astype('int')
bus_station_merge_CH_class_AB['CH거리구분_B'] = bus_station_merge_CH_class_AB['CH거리구분_B'].astype('int')

# 병합된 데이터 셋
bus_station_merge_CH_class_AB.head(1)


테이블간 결합됨 True
테이블간 결합됨 True


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,AD5,FD6,CE7,HP8,PM9,category_sum,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,8,0,1,0,17,0,0,5.0,25.0


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,AD5,FD6,CE7,HP8,PM9,category_sum,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,8,0,1,0,17,0,0,5,25


# 5. 버스 정류장과 하천과의 거리 갯수 구하기
위의 작업들과 동일하며, 버스정류장과 하천과의 거리를 구하여 구간별로 나눠 갯수를 센다

In [43]:
# ToDo. 버스정류장과하천과의거리 구하는 코드 추가하기

In [44]:
bus_stream = pd.read_pickle('./버스정류장과하천과의거리.pickle')
bus_stream.head(2)

,정류장번호,정류장명,location,A1,하천좌표,distance
0,ASB285000077,성환8리,"(36.92146807, 127.12277401)",64800004888020200241UJB4000001030,"(35.8484069199, 127.9088593132)",138.392944
1,ASB285000077,성환8리,"(36.92146807, 127.12277401)",64800004888020200241UJB4000001031,"(35.841851007, 127.9084686265)",139.003014


In [45]:
bus_stream_A  = bus_stream[(0<=bus_stream['distance'])&(bus_stream['distance']<=0.5)]
bus_stream_B  = bus_stream[(0.5<bus_stream['distance'])&(bus_stream['distance']<=1)]
bus_stream_A.head(2)
bus_stream_B.head(2)


# # class_A : 버스 정류장과 집,회사 사이의 거리가 0<=distance<=0.5
bus_stream_class_A_df= pd.DataFrame(bus_stream_A['정류장번호'].value_counts()).reset_index()
bus_stream_class_A_df.columns =['정류장번호','하천거리구분_A']
bus_stream_class_A_df.sort_values('정류장번호',ascending=True,inplace=True)
bus_stream_class_A_df.reset_index(drop=True,inplace=True)
bus_stream_class_A_df.head(2)

# # class_B : 버스 정류장과 집,회사 사이의 거리가 0.5<distance<=1
bus_stream_class_B_df= pd.DataFrame(bus_stream_B['정류장번호'].value_counts()).reset_index()
bus_stream_class_B_df.columns =['정류장번호','하천거리구분_B']
bus_stream_class_B_df.sort_values('정류장번호',ascending=True,inplace=True)
bus_stream_class_B_df.reset_index(drop=True,inplace=True)
bus_stream_class_B_df.head(2)

,정류장번호,정류장명,location,A1,하천좌표,distance
7736,ASB285000077,성환8리,"(36.92146807, 127.12277401)",64400004413020210447UJB1000000007,"(36.9202740809, 127.1269891067)",0.398270
37124,ASB285000094,성환8리,"(36.9214794, 127.1225049)",64400004413020210447UJB1000000007,"(36.9202740809, 127.1269891067)",0.421355


,정류장번호,정류장명,location,A1,하천좌표,distance
17532,ASB285000078,송덕,"(36.92378302, 127.11897568)",64400004413020210447UJB1000000007,"(36.9202740809, 127.1269891067)",0.813301
46920,ASB285000095,성환터미널,"(36.91722136, 127.13273235)",64400004413020210447UJB1000000007,"(36.9202740809, 127.1269891067)",0.613733


,정류장번호,하천거리구분_A
0,ASB285000077,1
1,ASB285000094,1


,정류장번호,하천거리구분_B
0,ASB285000078,1
1,ASB285000095,1


In [46]:
# 테이블을 병합하기위해, copy본과 결합
bus_station_merge_class_AB_cp = bus_station_merge_CH_class_AB.copy()

bus_station_merge_stream_class_A = pd.merge(bus_station_merge_class_AB_cp,bus_stream_class_A_df,on='정류장번호',how='left')

# 천안시의 버스정류장 데이터와, 천안시 버스정류장-공원거리구분 A 데이터가 잘 병합됬었는지 데이터 갯수를 통해 확인
print('테이블간 결합됨',len(bus_station)==len(bus_station_merge_stream_class_A))

bus_station_merge_stream_class_AB = pd.merge(bus_station_merge_stream_class_A,bus_stream_class_B_df, on='정류장번호',how='left')

print('테이블간 결합됨',len(bus_station)==len(bus_station_merge_stream_class_AB))

bus_station_merge_stream_class_AB.head(1)
# 테이블 병합이후, NaN값은 0으로 채우고, 병합한 컬럼의 자료형은 int 로 변경
bus_station_merge_stream_class_AB['하천거리구분_A'] = bus_station_merge_stream_class_AB['하천거리구분_A'].fillna(0)
bus_station_merge_stream_class_AB['하천거리구분_B'] = bus_station_merge_stream_class_AB['하천거리구분_B'].fillna(0)

bus_station_merge_stream_class_AB['하천거리구분_A'] = bus_station_merge_stream_class_AB['하천거리구분_A'].astype('int')
bus_station_merge_stream_class_AB['하천거리구분_B'] = bus_station_merge_stream_class_AB['하천거리구분_B'].astype('int')

bus_station_merge_stream_class_AB.head(1)


테이블간 결합됨 True
테이블간 결합됨 True


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,CE7,HP8,PM9,category_sum,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,1,0,17,0,0,5,25,1.0,NaN


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,CE7,HP8,PM9,category_sum,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,1,0,17,0,0,5,25,1,0


In [47]:
# 최종적으로 사용하게 될 데이터프레임
bus_station_merge_stream_class_AB.head()


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,CE7,HP8,PM9,category_sum,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,1,0,17,0,0,5,25,1,0
1,ASB285000078,송덕,36.923783,127.118976,0,2,2,1,0,2,...,0,0,0,12,0,0,11,15,0,1
2,ASB285000079,국립축산과학원,36.925931,127.111136,0,1,1,1,0,0,...,0,0,0,3,0,0,12,22,0,0
3,ASB285000094,성환8리,36.921479,127.122505,0,2,1,0,0,2,...,0,0,0,15,0,0,5,24,1,0
4,ASB285000095,성환터미널,36.917221,127.132732,1,11,4,3,30,3,...,0,16,0,280,0,1,11,16,0,1


# Score 계산하기

버스정류장의 반경 2km 이내에 있는 각 카테고리(마트,편의점,병원,,)의 수 를 집계하여

구분 A의 값은 2배의 값을 곱하여 더하고,

구분 B의 값은 자기 자신의 값을 더하여 score 칼럼을 구한다.

(score 칼럼 : 버스정류장의 중요도를 나타내는 정도)

In [48]:
score_df = bus_station_merge_stream_class_AB.copy()

In [49]:
# score 칼럼 구하기
score_df['score'] = score_df['category_sum'].astype('int') + (score_df['공원거리구분_A']*2 + score_df['공원거리구분_B']) + (score_df['CH거리구분_A'] *2 + score_df['CH거리구분_B']) + (score_df['하천거리구분_A'] *2 + score_df['하천거리구분_B'])
# score_df[['주변시설의총합','score']]
# sorted_score_df =score_df[['정류장번호','정류장명','위도','경도','도로명주소','시','구','읍/면/동','리','score']].sort_values(by='score',ascending=False)
# sorted_score_df
score_df.head(2)


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,HP8,PM9,category_sum,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B,score
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,1,0,17,0,0,5,25,1,0,54
1,ASB285000078,송덕,36.923783,127.118976,0,2,2,1,0,2,...,0,0,12,0,0,11,15,0,1,50


# 6. 클러스터링 진행하기

In [81]:
from sklearn.cluster import KMeans

data = score_df.copy()
coordinates = data[['위도', '경도']]
num_clusters = 21 # You can change this number
kmeans = KMeans(n_clusters=num_clusters,random_state=98)

kmeans.fit(coordinates)

data['cluster'] = kmeans.labels_


print("Cluster Centers:")
print(kmeans.cluster_centers_)
print(data['cluster'].value_counts())

data.head()


KMeans(n_clusters=21, random_state=98)

Cluster Centers:
[[ 36.88019216 127.19422124]
 [ 36.80102722 127.15148067]
 [ 36.72997027 127.24998554]
 [ 36.87543791 127.14079172]
 [ 36.67802031 127.06686907]
 [ 36.78032698 127.3789236 ]
 [ 36.84237469 127.26931611]
 [ 36.73325356 127.10528998]
 [ 36.76741516 127.20047689]
 [ 36.80151801 127.11735896]
 [ 36.8303058  127.1260509 ]
 [ 36.75147293 127.29788995]
 [ 36.68861566 127.127036  ]
 [ 36.83382793 127.17218414]
 [ 36.79910451 127.32965861]
 [ 36.92575295 127.19309217]
 [ 36.91261371 127.11593288]
 [ 36.90717371 127.23986794]
 [ 36.94825014 127.13077937]
 [ 36.78884473 127.23854679]
 [ 36.7548496  127.14817258]]
1     258
10    222
9     220
16    161
3     128
13    114
2     108
11    105
18     98
20     90
19     90
0      90
8      80
17     79
14     75
15     72
6      71
4      70
7      69
12     68
5      46
Name: cluster, dtype: int64


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,PM9,category_sum,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B,score,cluster
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,17,0,0,5,25,1,0,54,16
1,ASB285000078,송덕,36.923783,127.118976,0,2,2,1,0,2,...,0,12,0,0,11,15,0,1,50,16
2,ASB285000079,국립축산과학원,36.925931,127.111136,0,1,1,1,0,0,...,0,3,0,0,12,22,0,0,49,16
3,ASB285000094,성환8리,36.921479,127.122505,0,2,1,0,0,2,...,0,15,0,0,5,24,1,0,51,16
4,ASB285000095,성환터미널,36.917221,127.132732,1,11,4,3,30,3,...,0,280,0,1,11,16,0,1,320,16


## 최적의 클러스터 갯수 구하기

Elbow point와 silhouette_score 를 통해, 좌표를 통한 클러스터를 묶을 클러스터 갯수를 구하기 위해, 

다음 2가지 방법으로 최적의 클러스터 갯수를 구한다.

### 6.1 Elbow 확인

In [ ]:
data = score_df.copy()
coordinates = data[['위도', '경도']]
num_clusters_range = range(10, 31)  # You can change this range

inertias = []
for n in num_clusters_range:
    kmeans = KMeans(n_clusters=n, random_state=98)

    # Fit the model to the coordinates
    kmeans.fit(coordinates)
    inertias.append(kmeans.inertia_)

# Plot the elbow curve
plt.figure(figsize=(10, 6))
plt.plot(num_clusters_range, inertias, marker='o')
plt.title('Elbow Curve')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.show()


# # Add a new column to your DataFrame with the cluster labels
# data['cluster'] = kmeans.labels_

# # Print the cluster centers (the centroids)

# '''
# 센트럴 포인트에서 가까운 정류장만 고려하기?
# '''
# print("Cluster Centers:")
# print(kmeans.cluster_centers_)
# print(data['cluster'].value_counts())

# # Print the first few rows of the DataFrame with the cluster labels
# data.head()



### 6.2 실루엣 스코어 확인

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

data = score_df.copy()
coordinates = data[['위도', '경도']]
num_clusters_range = range(10, 31)  # You can change this range

inertias = []
silhouette_scores = []

for n in num_clusters_range:
    kmeans = KMeans(n_clusters=n, random_state=98)

    # Fit the model to the coordinates
    kmeans.fit(coordinates)
    inertias.append(kmeans.inertia_)

    # Calculate silhouette score
    labels = kmeans.labels_
    silhouette_avg = silhouette_score(coordinates, labels)
    silhouette_scores.append(silhouette_avg)

# Plot the elbow curve
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(num_clusters_range, inertias, marker='o')
plt.title('Elbow Curve')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')

# Plot the silhouette scores
plt.subplot(1, 2, 2)
plt.plot(num_clusters_range, silhouette_scores, marker='o', color='r')
plt.title('Silhouette Score')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')

plt.tight_layout()
plt.show()


In [ ]:
def visualize_silhouette(cluster_lists, X_features): 

    from sklearn.datasets import make_blobs
    from sklearn.cluster import KMeans
    from sklearn.metrics import silhouette_samples, silhouette_score

    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    import math

    # 입력값으로 클러스터링 갯수들을 리스트로 받아서, 각 갯수별로 클러스터링을 적용하고 실루엣 개수를 구함
    n_cols = len(cluster_lists)

    # plt.subplots()으로 리스트에 기재된 클러스터링 수만큼의 sub figures를 가지는 axs 생성 
    fig, axs = plt.subplots(figsize=(4*n_cols, 4), nrows=1, ncols=n_cols)

    # 리스트에 기재된 클러스터링 갯수들을 차례로 iteration 수행하면서 실루엣 개수 시각화
    for ind, n_cluster in enumerate(cluster_lists):

        # KMeans 클러스터링 수행하고, 실루엣 스코어와 개별 데이터의 실루엣 값 계산. 
        clusterer = KMeans(n_clusters = n_cluster, max_iter=500, random_state=0)
        cluster_labels = clusterer.fit_predict(X_features)

        sil_avg = silhouette_score(X_features, cluster_labels)
        sil_values = silhouette_samples(X_features, cluster_labels)

        y_lower = 10
        axs[ind].set_title('Number of Cluster : '+ str(n_cluster)+'\n' \
                          'Silhouette Score :' + str(round(sil_avg,3)) )
        axs[ind].set_xlabel("The silhouette coefficient values")
        axs[ind].set_ylabel("Cluster label")
        axs[ind].set_xlim([-0.1, 1])
        axs[ind].set_ylim([0, len(X_features) + (n_cluster + 1) * 10])
        axs[ind].set_yticks([])  # Clear the yaxis labels / ticks
        axs[ind].set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1])

        # 클러스터링 갯수별로 fill_betweenx( )형태의 막대 그래프 표현. 
        for i in range(n_cluster):
            ith_cluster_sil_values = sil_values[cluster_labels==i]
            ith_cluster_sil_values.sort()

            size_cluster_i = ith_cluster_sil_values.shape[0]
            y_upper = y_lower + size_cluster_i

            color = cm.nipy_spectral(float(i) / n_cluster)
            axs[ind].fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_sil_values, \
                                facecolor=color, edgecolor=color, alpha=0.7)
            axs[ind].text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
            y_lower = y_upper + 10

        axs[ind].axvline(x=sil_avg, color="red", linestyle="--")
visualize_silhouette([10], coordinates)


# 7. 시각화

클러스터링한 결과를 통해, 따릉이를 거치할 버스 정류장의 후보군을 선정합니다.

In [83]:
data.head()

,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,PM9,category_sum,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B,score,cluster
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,17,0,0,5,25,1,0,54,16
1,ASB285000078,송덕,36.923783,127.118976,0,2,2,1,0,2,...,0,12,0,0,11,15,0,1,50,16
2,ASB285000079,국립축산과학원,36.925931,127.111136,0,1,1,1,0,0,...,0,3,0,0,12,22,0,0,49,16
3,ASB285000094,성환8리,36.921479,127.122505,0,2,1,0,0,2,...,0,15,0,0,5,24,1,0,51,16
4,ASB285000095,성환터미널,36.917221,127.132732,1,11,4,3,30,3,...,0,280,0,1,11,16,0,1,320,16


## potly를 통한 시각화

In [111]:
bus_station_Main.columns

Index(['정류장번호', '정류장명', '위도', '경도', '정보수집일시', '모바일단축번호', '도시코드', '도시명',
       '관리도시명', 'MT1', 'CS2', 'PS3', 'SC4', 'AC5', 'PK6', 'OL7', 'SW8', 'BK9',
       'CT1', 'AG2', 'PO3', 'AT4', 'AD5', 'FD6', 'CE7', 'HP8', 'PM9',
       '주변시설의총합', 'coord2address', 'region_2depth_name', '도로명주소', '시', '구',
       '읍/면/동', '리'],
      dtype='object')

In [114]:
data_cp = data.copy()
data_cp.sort_values(by = 'score',inplace=True,ascending=False)
data_bus_station_merge = pd.merge(data_cp,bus_station_Main[['정류장번호','도로명주소']],on='정류장번호',how='left')

data_bus_station_merge = data_bus_station_merge.drop_duplicates(subset='정류장명')

data_bus_station_merge = data_bus_station_merge.head(188)
data_bus_station_merge.head(2)

,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B,score,cluster,for_marker_size,도로명주소
0,CAB285000936,성정중학교,36.830708,127.137961,1,66,1,1,25,27,...,5,10,4,47,0,0,1220,10,10.0,충남 천안시 서북구 두정동 1801
1,ASB288001967,불당상업지구입구,36.811821,127.108607,1,27,11,0,270,36,...,2,7,0,3,1,0,1217,9,10.0,충남 천안시 서북구 불당동 722


In [118]:
data_bus_station_merge['cluster'].value_counts()

10    75
9     60
1     42
3     10
13     1
Name: cluster, dtype: int64

In [119]:
# 마커 사이즈를 위한, 마커 크기 구하기
min_score = data_bus_station_merge['score'].min()
max_score = data_bus_station_merge['score'].max()
data_bus_station_merge['for_marker_size'] = 1 + ((data_bus_station_merge['score'] - min_score) / (max_score - min_score)) * 9
data_bus_station_merge['for_marker_size'] = data_bus_station_merge['for_marker_size'].round()

In [120]:
import plotly.express as px


df = data_bus_station_merge.copy()
# Create a scatter map using Plotly Express
fig = px.scatter_mapbox(
    df,
    lat='위도',
    lon='경도',
    color='cluster',  # Color points based on the cluster column
    size_max=20,      # Adjust point size
    zoom=9,           # Initial map zoom level
    hover_name='정류장명',
    size='for_marker_size'
)

# Customize the map layout
# https://plotly.com/python/mapbox-layers/
map_style = ["open-street-map", "carto-positron", "carto-darkmatter", "stamen-terrain", "stamen-toner" ,"stamen-watercolor" ]
fig.update_layout(mapbox_style=map_style[0])  # Choose a map style
fig.update_layout(
    margin={'l': 0, 'r': 0, 't': 0, 'b': 0},  # Adjust margin
    mapbox=dict(center=dict(lat=36.83682, lon=127.2077)),  # Set initial center
)

# Show the plot
# fig.show()


In [124]:
import os

# Replace characters that are not allowed in filenames
when_sanitized = when.replace(':', '-')

path = os.path.join(r'E:\제출용데이터dir\plotly시각화', f'{when_sanitized}.html')
pio.write_html(fig, path)


In [100]:
bus_bus = pd.read_pickle('./정류장간의거리.pickle')
bus_bus

,정류장번호,정류장명,location,정류장번호2,정류장명2,distance
0,ASB285000077,[성환8리],"[[36.92146807, 127.12277401]]",ASB285000077,[성환8리],0.000000
1,ASB285000077,[성환8리],"[[36.92146807, 127.12277401]]",ASB285000078,[송덕],0.424900
2,ASB285000077,[성환8리],"[[36.92146807, 127.12277401]]",ASB285000079,[국립축산과학원],1.149206
3,ASB285000077,[성환8리],"[[36.92146807, 127.12277401]]",ASB285000094,[성환8리],0.024011
4,ASB285000077,[성환8리],"[[36.92146807, 127.12277401]]",ASB285000095,[성환터미널],1.004732
...,...,...,...,...,...,...
5354591,GGB285500030,[천안종합터미널],"[[36.8198, 127.1556]]",GGB285000034,[성환터미널],10.929045
5354592,GGB285500030,[천안종합터미널],"[[36.8198, 127.1556]]",GGB285000035,[남서울대학],10.110406
5354593,GGB285500030,[천안종합터미널],"[[36.8198, 127.1556]]",GGB285500010,[천안TG.단대.천안대],0.721364
5354594,GGB285500030,[천안종합터미널],"[[36.8198, 127.1556]]",GGB285500020,[천안종합터미널],0.012454


# 전처리 실험과정

In [102]:
bus_bus_cp = bus_bus[bus_bus['distance']<=0.5]
bus_bus_cp.to_pickle('./정류장간의거리500m.pickle')

In [103]:
bus_bus_cp
df


,정류장번호,정류장명,location,정류장번호2,정류장명2,distance
0,ASB285000077,[성환8리],"[[36.92146807, 127.12277401]]",ASB285000077,[성환8리],0.000000
1,ASB285000077,[성환8리],"[[36.92146807, 127.12277401]]",ASB285000078,[송덕],0.424900
3,ASB285000077,[성환8리],"[[36.92146807, 127.12277401]]",ASB285000094,[성환8리],0.024011
101,ASB285000077,[성환8리],"[[36.92146807, 127.12277401]]",ASB285001067,[송덕],0.431416
221,ASB285000077,[성환8리],"[[36.92146807, 127.12277401]]",CAB285000077,[성환8리],0.003005
...,...,...,...,...,...,...
5354484,GGB285500030,[천안종합터미널],"[[36.8198, 127.1556]]",CAB285071724,[신부청광아파트],0.373937
5354502,GGB285500030,[천안종합터미널],"[[36.8198, 127.1556]]",CAB285071752,[고속터미널],0.127266
5354512,GGB285500030,[천안종합터미널],"[[36.8198, 127.1556]]",CAB285071768,[우방아이유셀아파트],0.484774
5354594,GGB285500030,[천안종합터미널],"[[36.8198, 127.1556]]",GGB285500020,[천안종합터미널],0.012454


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,category_sum,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B,score,cluster,for_marker_size
856,CAB285000936,성정중학교,36.830708,127.137961,1,66,1,1,25,27,...,1145,5,10,4,47,0,0,1220,10,10.0
174,ASB288001967,불당상업지구입구,36.811821,127.108607,1,27,11,0,270,36,...,1201,2,7,0,3,1,0,1217,9,10.0
857,CAB285000937,성정중학교,36.829957,127.137722,2,61,1,1,16,25,...,1098,5,9,3,60,0,0,1183,10,10.0
855,CAB285000935,두정양돈농협,36.833573,127.138038,1,54,9,3,70,19,...,1085,5,7,7,28,0,0,1144,10,9.0
924,CAB285001024,두정도서관,36.833743,127.135533,1,49,13,5,79,18,...,1069,7,6,8,23,0,0,1128,10,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852,CAB285000932,"부성2동행정복지센터,두정경남아파트",36.837636,127.138297,1,25,19,5,75,4,...,476,3,9,9,14,0,0,523,10,1.0
935,CAB285001035,두정역,36.832665,127.147905,0,24,28,1,51,8,...,441,2,7,9,48,0,0,518,10,1.0
69,ASB285000908,성정1동주민센터,36.818040,127.142661,1,16,9,1,18,13,...,301,7,13,55,80,0,0,518,1,1.0
186,ASB288010773,천안세관,36.823358,127.143937,2,24,8,1,12,10,...,352,6,11,36,70,0,0,517,10,1.0
